In [1]:
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, roc_curve, auc
import matplotlib.pylab as plt

from dmba import regressionSummary, classificationSummary
from dmba import liftChart, gainsChart

no display found. Using non-interactive Agg backend


In [3]:
df = pd.read_csv("ToyotaCorolla.csv")
df.head(2)

,model,price,age_08_04,mfg_month,mfg_year,km,fuel_type,hp,met_color,color,...,powered_windows,power_steering,radio,mistlamps,sport_model,backseat_divider,metallic_rim,radio_cassette,parking_assistant,tow_bar
0,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13500,23,10,2002,46986,Diesel,90,1,Blue,...,1,1,0,0,0,1,0,0,0,0
1,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13750,23,10,2002,72937,Diesel,90,1,Silver,...,0,1,0,0,0,1,0,0,0,0


In [5]:
df.columns

Index(['model', 'price', 'age_08_04', 'mfg_month', 'mfg_year', 'km',
       'fuel_type', 'hp', 'met_color', 'color', 'automatic', 'cc', 'doors',
       'cylinders', 'gears', 'quarterly_tax', 'weight', 'mfr_guarantee',
       'bovag_guarantee', 'guarantee_period', 'abs', 'airbag_1', 'airbag_2',
       'airco', 'automatic_airco', 'boardcomputer', 'cd_player',
       'central_lock', 'powered_windows', 'power_steering', 'radio',
       'mistlamps', 'sport_model', 'backseat_divider', 'metallic_rim',
       'radio_cassette', 'parking_assistant', 'tow_bar'],
      dtype='object')

In [8]:
# Part of data preparation and data cleaning
excludeColumns = ('price','id', 'model', 'fuel_type', 'color')
predictor = []
for s in df.columns:
    if s not in excludeColumns:
        predictor.append(s)

outcome = 'price'

In [9]:
print(predictor)

['age_08_04', 'mfg_month', 'mfg_year', 'km', 'hp', 'met_color', 'automatic', 'cc', 'doors', 'cylinders', 'gears', 'quarterly_tax', 'weight', 'mfr_guarantee', 'bovag_guarantee', 'guarantee_period', 'abs', 'airbag_1', 'airbag_2', 'airco', 'automatic_airco', 'boardcomputer', 'cd_player', 'central_lock', 'powered_windows', 'power_steering', 'radio', 'mistlamps', 'sport_model', 'backseat_divider', 'metallic_rim', 'radio_cassette', 'parking_assistant', 'tow_bar']


In [13]:
# Partition Data

x = df[predictor]
y = df[outcome]

train_x, valid_x, train_y, valid_y = train_test_split(x,y,test_size = 0.4, random_state = 1) 

In [14]:
# Train Linear Regression model

reg = LinearRegression()
reg.fit(train_x, train_y)

LinearRegression()

In [15]:
# Evaluate Performance training

regressionSummary?


Signature: regressionSummary(y_true: Any, y_pred: Any) -> None
Docstring:
print regression performance metrics

Input:
    y_true: actual values
    y_pred: predicted values
File:      /Applications/anaconda3/lib/python3.12/site-packages/dmba/metric.py
Type:      function

In [16]:
# Evaluate Performance training
regressionSummary(train_y, reg.predict(train_x))
regressionSummary(valid_y, reg.predict(valid_x))



Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 1121.0606
            Mean Absolute Error (MAE) : 811.6770
          Mean Percentage Error (MPE) : -0.8630
Mean Absolute Percentage Error (MAPE) : 8.0054

Regression statistics

                      Mean Error (ME) : 97.1891
       Root Mean Squared Error (RMSE) : 1382.0352
            Mean Absolute Error (MAE) : 880.1396
          Mean Percentage Error (MPE) : 0.0138
Mean Absolute Percentage Error (MAPE) : 8.8744
